In [6]:
import math
import numpy as np

In [1]:
# Take in mean and variance, return probability
def multivariate_gaussian_pdf(x, mean, cov_mat):
    p = len(cov_mat)
    left_term = 1 / math.sqrt(np.linalg.det(cov_mat) * (2 * math.pi) ** p)
    right_term = np.e ** (-0.5 * )
    return left_term * right_term



In [58]:
def Estep(X, means, cov_mat, G):
    A = X.T
    p = len(cov_mat)
    probs = np.zeros((X.shape[0], G))
    for k, mu in enumerate(means):
        probs[:,k] = np.sum((np.linalg.inv(cov_mat) @ (X - mu).T) * (X - mu).T, axis=0)
    probs = probs * -0.5
    probs = np.exp(probs)
    left_term = 1 / math.sqrt(np.linalg.det(cov_mat) * (2 * math.pi) ** p)
    probs = probs * left_term
    return probs